In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Turn off GPU
if False:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow import keras
import tensorflow as tf
from glob import glob
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')

print(tf.__version__)
print(gpus)

if len(gpus) > 0:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 2))])

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from glob import glob

models = np.array(sorted(glob("model-saved\\deeplabV3+_mobileNetV2\\*")))

index_model_0_5 = np.char.find(models, "_alpha_0.5") != -1

models_0_5 = models[index_model_0_5]
models_1_0 = models[~index_model_0_5]

models_0_5, models_1_0

(array(['model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_0.5_lr_0.0001_regularizer_l1_0.0001_l2_1e-05',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_0.5_lr_0.0001_regularizer_l1_1e-05_l2_1e-05',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_0.5_lr_0.001_regularizer_l1_0.0001_l2_1e-05',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_0.5_lr_0.001_regularizer_l1_0.0001_l2_1e-06',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_0.5_lr_0.001_regularizer_l1_1e-05_l2_1e-05'],
       dtype='<U98'),
 array(['model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_1.0_lr_0.0001_regularizer_l1_0.0001_l2_1e-05',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_1.0_lr_0.0001_regularizer_l1_0.0001_l2_1e-06',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_1.0_lr_0.0001_regularizer_l1_1e-05_l2_1e-05',
        'model-saved\\deeplabV3+_mobileNetV2\\deeplayer_12_alpha_1.0_lr_0.0001_regulari

In [13]:
modelFolder = models_1_0[0]

In [14]:
from models import deeplabV3

imageSize = (256, 256)
alpha = float(modelFolder[modelFolder.find("alpha_")+6: modelFolder.find("alpha_")+9])
mobileLayers = {"shallowLayer": "block_2_project_BN",
                "deepLayer": "block_12_project_BN"}

model = deeplabV3(imageSize=imageSize, nClasses=2, alpha=alpha, withArgmax=True, mobileLayers=mobileLayers)

In [15]:
fold = 0

model.load_weights(modelFolder + f"\\fold_{fold}.ckpt")

In [16]:
from utils.model_utils import BatchGenerator

images = glob("utils\\split\\TrainVal\\img\\*")

print(len(images))

representativeDatasetGen = BatchGenerator(images)

2975


In [17]:
from utils.model_utils import toTFlite
from tensorflow import lite as tflite

toTFlite(
    model=model,
    savePath=modelFolder + f"\\fold_{fold}",
    representativeDatasetGen=representativeDatasetGen,
    supportedOps=[
        tflite.OpsSet.SELECT_TF_OPS,
        tflite.OpsSet.TFLITE_BUILTINS])

INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmpbepej2t2\assets


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmpbepej2t2\assets
C:\Users\rafae\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
